In [ ]:
import pandas as pd
import os
import numpy as np
import sys

In [ ]:
# this is a sheet of labor data received
labor = pd.read_excel("labordata.xlsx", header=0)

In [ ]:
# this is the qty and fulfillment status by MO
modata = pd.read_excel("modata.xlsx", header=0)

In [ ]:
# creating a total labor column for each line
labor['minConvert'] = labor['mins']/60
labor['timeCalc'] = labor['hours'] + labor['minConvert']

In [ ]:
# only need finished MOs
completeLabor = labor[labor['Status'] == 'Done'].copy()

In [ ]:
completeLabor.drop(['hours','mins','Status','minConvert'], axis=1, inplace=True)

In [ ]:
# summing the useful columns
completeLabor = completeLabor.groupby(['MO','Part']).sum()
modata = modata.groupby(['MO','Part']).sum()
completeLabor.reset_index(inplace=True)
modata.reset_index(inplace=True)

In [ ]:
# creates a new dataFrame with both labor used per order and qty of product created
combinedData = pd.DataFrame()
for part in completeLabor['Part'].unique():
    laborPart = completeLabor[completeLabor['Part'] == part].copy()
    moPart = modata[modata['Part'] == part].copy()
    if len(moPart) > 0:
        for mo in laborPart['MO'].unique():
            laborMO = laborPart[laborPart['MO'] == mo].copy()
            moMO = moPart[moPart['MO'] == mo].copy()
            if len(moMO) > 0:
                holdingDF = pd.merge(laborMO.copy(), moMO[['Part','QtyFulf']].copy(), how='left', on='Part')
                combinedData = combinedData.copy().append(holdingDF.copy())
            else:
                print(mo, part)
    else:
        print(part)



In [ ]:
# calculate the labor per panel
combinedData['HoursPerUnit'] = combinedData['timeCalc'] / combinedData['QtyFulf']

In [ ]:
# save it
writer = pd.ExcelWriter('result.xlsx')
combinedData.to_excel(writer, sheet_name='sheet', index=False)
writer.save()